In diesem Notebook möchte ich die autograd funktion von pytorch anhand eines kleinen Netzwerkes testen.

In [9]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd.functional import jacobian
from torch.autograd import grad

Erstellung des einfachen Netzwerkes

In [10]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # Schichten festlegen
        self.Layer_1 = nn.Linear(3, 16)
        self.Output_Layer = nn.Linear(16, 2)

        # Aktivierungsfunktion definieren
        self.activation_fnc = nn.ReLU()

    def forward(self, x):
        x = self.Layer_1(x)
        x = self.activation_fnc(x)
        x = self.Output_Layer(x)

        return x

test_network = SimpleNN()

Ein paar einfache werte austesten (mit Gradientenverfolgung des Eingangs)

In [11]:
test_input = torch.tensor([1, 2, 1.5], dtype=torch.float32, requires_grad=True)
print('Test Input: \n', test_input)

test_output = test_network(test_input)
print('Ergebnis: \n', test_output)

Test Input: 
 tensor([1.0000, 2.0000, 1.5000], requires_grad=True)
Ergebnis: 
 tensor([0.8292, 0.1727], grad_fn=<ViewBackward0>)


Gradienten bezüglich der Eingangsgrößen berechnen (man kann den Gradienten nur bezüglich einer skalaren Ausgangsgröße berechnen)

In [12]:
J = jacobian(lambda inp: test_network(inp), test_input, create_graph=True)  # lambda ist in diesem fall einfach eine kleine Funktion (nur nicht über def), da jacobian eine Funktion als eingabe erwartet. Die Funktion wird dann mit dem test_input ausgewertet und der Gradient wird erstellt.
print('Jacobimatrix berechnen: \n', J)

Jacobimatrix berechnen: 
 tensor([[ 0.1909,  0.3073,  0.0489],
        [-0.0410,  0.0945,  0.1398]], grad_fn=<ViewBackward0>)


In [19]:
gradient = grad(test_output, test_input, grad_outputs=torch.ones_like(test_output), create_graph=True)

print('Berechnung Gradient mit Grad: \n', gradient)

Berechnung Gradient mit Grad: 
 (tensor([0.1499, 0.4018, 0.1887], grad_fn=<ViewBackward0>),)


Prüfen, ob die backward() funktion auch auf Basis der Jacobimatrix funktioniert.

In [14]:
test_loss = J.sum()
print('Test Loss: \n', test_loss)
print()

test_loss.backward()
print('Gradient des Test Loss nach eingängen: \n', test_input.grad)
print()

name_param = list(test_network.named_parameters())
print(name_param[0][0], ': \n', name_param[0][1].grad)
print()
print(name_param[1][0], ': \n', name_param[1][1].grad)


Test Loss: 
 tensor(0.7404, grad_fn=<SumBackward0>)

Gradient des Test Loss nach eingängen: 
 tensor([0., 0., 0.])

Layer_1.weight : 
 tensor([[ 0.1466,  0.1466,  0.1466],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0313,  0.0313,  0.0313],
        [ 0.1104,  0.1104,  0.1104],
        [ 0.1408,  0.1408,  0.1408],
        [-0.0292, -0.0292, -0.0292],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.1505,  0.1505,  0.1505],
        [ 0.2284,  0.2284,  0.2284],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.4545,  0.4545,  0.4545],
        [ 0.0000,  0.0000,  0.0000]])

Layer_1.bias : 
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


Anmerkung: Wenn keine aktivierungsfunktion verwendet wird, passiert folgendes:

-Gradient des Test Loss bezüglich der Eingänge ist None
    
-Gradient des bias bezüglich der Eingänge ist None

Das liegt daran, dass diese Einträge beim Ableiten herausfliegen. (hiernach fehlt etwas - siehe chatgpt oder andere skripte)